In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
user_df = pd.read_csv("data/userTable.csv", parse_dates=['Signup Date'])

user_df.shape

(34050, 3)

In [3]:
user_df.head()

,UserID,Signup Date,User Segment
0,U133159,2018-04-14 07:01:16.202607+00:00,C
1,U129368,2017-12-02 09:38:41.584270+00:00,B
2,U109654,2013-03-19 11:38:55+00:00,B
3,U108998,2018-01-18 08:29:51.627954+00:00,C
4,U131393,2018-03-27 08:05:28.806800+00:00,B


In [3]:
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34050 entries, 0 to 34049
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   UserID        34050 non-null  object             
 1   Signup Date   34050 non-null  datetime64[ns, UTC]
 2   User Segment  34050 non-null  object             
dtypes: datetime64[ns, UTC](1), object(2)
memory usage: 798.2+ KB


In [4]:
visitorsLog_df = pd.read_csv('data/VisitorLogsData.csv', parse_dates=['VisitDateTime'])

In [5]:
visitorsLog_df.shape

(6588000, 9)

In [6]:
visitorsLog_df.head()

,webClientID,VisitDateTime,ProductID,UserID,Activity,Browser,OS,City,Country
0,WI10000050298,2018-05-07 04:28:45.970,pr100631,NaN,NaN,Chrome Mobile,Android,Chennai,India
1,WI10000025922,2018-05-13 07:26:04.964,pr100707,NaN,NaN,Chrome,Windows,NaN,Taiwan
2,WI100000204522,2018-05-11 11:43:42.832,pr100030,NaN,click,Chrome,windows,Gurgaon,India
3,WI10000011974,2018-05-13 15:20:23.436,Pr100192,NaN,CLICK,Chrome,Windows,,
4,WI100000441953,2018-05-08 20:44:25.238,Pr100762,NaN,click,Chrome,mac os x,Iselin,United States


In [7]:
visitorsLog_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6588000 entries, 0 to 6587999
Data columns (total 9 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   webClientID    object
 1   VisitDateTime  object
 2   ProductID      object
 3   UserID         object
 4   Activity       object
 5   Browser        object
 6   OS             object
 7   City           object
 8   Country        object
dtypes: object(9)
memory usage: 452.4+ MB


In [8]:
sample_df = pd.read_csv("data/sample_submission_M7Vpb9f.csv")
sample_df.shape

(34050, 9)

In [9]:
visitorsLog_df = visitorsLog_df.dropna(subset=['UserID'])

visitorsLog_df.shape

(650695, 9)

In [10]:
visitdatetime = []
for i in visitorsLog_df['VisitDateTime'].fillna(''):
    if '-' in i:
        dates = i.split()[0]
        times = i.split()[-1].split('.')[0]
        visitdatetime.append(dates+" "+times)
    else: visitdatetime.append(i)

visitorsLog_df['newVisitDateTime'] = visitdatetime

In [11]:
# convert from unixtimestamp to actual datetime
def convert_2_timestamp(column, data):
    """convert from unix time to readable timestamp
        args: column: columns that needs to be converted to timestamp
                data: data that has the specified column
    """
    if column in data.columns.values:
        timestamp_ = []
        for time_unix in data[column]:
            if '-' in str(time_unix):
                timestamp_.append(time_unix)
            elif len(str(time_unix)) == 19:
                a = datetime.datetime.fromtimestamp(np.int64(time_unix[:10]))
                timestamp_.append(a.strftime('%Y-%m-%d %H:%M:%S'))      
            else: timestamp_.append(time_unix)
        
        return timestamp_
                                  
    else: print(f"{column} not in data")


In [12]:
visitorsLog_df['newVisitDateTime'] = convert_2_timestamp("newVisitDateTime", visitorsLog_df)


In [13]:
# take last 21 days of visitors table
visitorsLog_df['newVisitDateTime'] = pd.to_datetime(visitorsLog_df['newVisitDateTime'].fillna(0))

visitorsLog_df['days'] = visitorsLog_df['newVisitDateTime'].dt.day

In [14]:
visitorsLog_df.head()

,webClientID,VisitDateTime,ProductID,UserID,Activity,Browser,OS,City,Country,newVisitDateTime,days
14,WI100000112772,1526366895249000000,Pr100017,U106593,click,Chrome Mobile,Android,,,2018-05-15 07:48:15,15.0
21,WI1000009977,1527058921790000000,Pr101008,U108297,NaN,Chrome Mobile,Android,Delhi,India,2018-05-23 08:02:01,23.0
23,WI100000223826,2018-05-10 06:28:53.391,Pr100241,U132443,NaN,Firefox,Windows,NaN,India,2018-05-10 06:28:53,10.0
24,WI10000021998,2018-05-08 12:40:02.153,pr100495,U134616,click,Chrome,Windows,Cottage Grove,United States,2018-05-08 12:40:02,8.0
33,WI10000036281,2018-05-11 15:35:43.689,Pr100363,U130784,click,Chrome,Chrome OS,New Delhi,India,2018-05-11 15:35:43,11.0


In [15]:
last_21_days_df = visitorsLog_df[visitorsLog_df['days'] != 28.0 ]

In [16]:
last_21_days_df.shape

(650683, 11)

In [17]:
# map last 21 days to user

combined_df = last_21_days_df.merge(user_df, on='UserID', how='inner')

combined_df.shape

(650683, 13)

## Clicks_last_7_days

 	

Count of Clicks in the last 7 days  by the user

In [18]:
activity_map = {"click": 'CLICK', "pageload":'PAGELOAD'}
combined_df['Activity'] = combined_df['Activity'].replace(activity_map)
combined_df['Activity'].value_counts()

CLICK       395608
PAGELOAD    167319
Name: Activity, dtype: int64

In [19]:
from datetime import datetime, date, timedelta
def last7(datestr):
    orig = datetime.strptime(datestr, "%Y-%m-%d")
    plus7 = orig-timedelta(7)
    return plus7

In [20]:
import datetime
last7_df = combined_df[combined_df['newVisitDateTime'] >
                       combined_df['newVisitDateTime'].max() - datetime.timedelta(days=7)]

last7_df.shape

(212619, 13)

In [22]:
count_last7 = last7_df[last7_df['Activity'] == 'CLICK'].groupby(['UserID']).agg({'Activity': 'count'}).reset_index()

In [23]:
count_last7.columns = ['UserID', 'Clicks_last_7_days']
count_last7.head()

,UserID,Clicks_last_7_days
0,U100003,2
1,U100008,23
2,U100009,4
3,U100012,14
4,U100013,3


In [24]:
my_sub = pd.DataFrame()
my_sub['UserID'] = sample_df.UserID

my_sub = my_sub.merge(count_last7, on='UserID', how='left')

my_sub.head()

,UserID,Clicks_last_7_days
0,U100002,NaN
1,U100003,2.0
2,U100004,NaN
3,U100005,NaN
4,U100006,NaN


In [25]:
my_sub['Clicks_last_7_days'] = my_sub['Clicks_last_7_days'].fillna(my_sub['Clicks_last_7_days'].mean())

### Pageloads_last_7_days
	

Count of Page loads in the last 7 days by the user

In [26]:
pageload_last7 = last7_df[last7_df['Activity'] == 'PAGELOAD'].groupby(['UserID']).agg({'Activity': 'count'}).reset_index()

In [27]:
pageload_last7.columns = ['UserID', 'Pageloads_last_7_days']
pageload_last7.head()

,UserID,Pageloads_last_7_days
0,U100003,1
1,U100004,1
2,U100005,1
3,U100006,1
4,U100008,21


In [28]:
my_sub = my_sub.merge(pageload_last7, on='UserID', how='left')
my_sub['Pageloads_last_7_days'] = my_sub['Pageloads_last_7_days'].fillna(my_sub['Pageloads_last_7_days'].mean())

### Most_Active_OS

 	

Most Frequently used OS by user. 

In [29]:
map_os = {'windows':'Windows', 'mac os x':'Mac OS X', 'linux':'Linux', 'ios':'IOS',
          'fedora':'Fedora', 'tizen':'Tizen', 'ubuntu':'Ubuntu', 'android':'Android', 'chrome os':'Chrome OS'}


combined_df['OS'] = combined_df['OS'].replace(map_os)

In [30]:
most_active_os = combined_df.groupby('UserID')['OS'].value_counts().reset_index(name='Count')

In [31]:
my_sub = my_sub.merge(most_active_os[['UserID', 'OS']], on='UserID', how='left')

In [32]:
my_sub['OS'] = my_sub['OS'].fillna(my_sub['OS'].mode()[0])

### Recently_Viewed_Product

Most recently viewed (page loads) product by the user.

If a user has not viewed any product then put it as Product101.

In [33]:
recent_df = combined_df[combined_df['newVisitDateTime'] > "2018-05-26 00:00:00"]
recent_df.shape

(50732, 13)

In [34]:
prod = recent_df[recent_df['Activity'] == 'PAGELOAD'].groupby('UserID')['ProductID'].value_counts().reset_index(name='Count')

prod.head()

,UserID,ProductID,Count
0,U100005,Pr100234,1
1,U100008,pr100089,2
2,U100008,pr100606,2
3,U100008,Pr100025,1
4,U100008,Pr100234,1


In [35]:
my_sub = my_sub.merge(prod[['UserID', 'ProductID']], on='UserID', how='left').fillna('Product101')
my_sub.head()

,UserID,Clicks_last_7_days,Pageloads_last_7_days,OS,ProductID
0,U100002,12.293281,3.872105,Android,Product101
1,U100003,2.000000,1.000000,Windows,Product101
2,U100004,12.293281,1.000000,Windows,Product101
3,U100005,12.293281,1.000000,Android,Pr100234
4,U100006,12.293281,1.000000,Android,Product101


### Most_Viewed_product_15_Days
	

Most frequently viewed (page loads) product by the user in the last 15 days. If there are multiple products that have a similar number of page loads then , consider the recent one. If a user has not viewed any product in the last 15 days then put it as Product101. 

In [36]:
last15_df = combined_df[combined_df['newVisitDateTime'] >
                       combined_df['newVisitDateTime'].max() - datetime.timedelta(days=15)]

pageload_last15 = last15_df[last15_df['Activity'] == 'PAGELOAD'].groupby(['UserID']).agg({'ProductID': 'value_counts'})

pageload_last15.head()

ProductID
UserID  ProductID           
U100002 Pr100258           2
U100003 Pr100079           1
U100004 pr101505           2
        Pr100069           1
        Pr100312           1

In [37]:
m_prod = pd.DataFrame(pageload_last15)
m_prod.columns = ['ProductCount']
m_prod = m_prod.reset_index()
m_prod.columns = ['UserID', 'Most_Viewed_product_15_Days', 'ProductCount']

In [38]:
m_prod.drop_duplicates(subset=['UserID'], inplace=True)
m_prod.shape

(21661, 3)

In [39]:
my_sub = my_sub.merge(m_prod[['UserID', 'Most_Viewed_product_15_Days']], on='UserID', how='left').fillna('Product101')
my_sub.head()

,UserID,Clicks_last_7_days,Pageloads_last_7_days,OS,ProductID,Most_Viewed_product_15_Days
0,U100002,12.293281,3.872105,Android,Product101,Pr100258
1,U100003,2.000000,1.000000,Windows,Product101,Pr100079
2,U100004,12.293281,1.000000,Windows,Product101,pr101505
3,U100005,12.293281,1.000000,Android,Pr100234,Pr100234
4,U100006,12.293281,1.000000,Android,Product101,Pr101111


### Simple Submission

In [43]:
my_sub.head()

,UserID,Clicks_last_7_days,Pageloads_last_7_days,OS,ProductID,Most_Viewed_product_15_Days
0,U100002,12.293281,3.872105,Android,Product101,Pr100258
1,U100003,2.000000,1.000000,Windows,Product101,Pr100079
2,U100004,12.293281,1.000000,Windows,Product101,pr101505
3,U100005,12.293281,1.000000,Android,Pr100234,Pr100234
4,U100006,12.293281,1.000000,Android,Product101,Pr101111


In [40]:
sample_df['Most_Viewed_product_15_Days'] = my_sub['Most_Viewed_product_15_Days']
sample_df['Most_Active_OS'] = my_sub['OS']
sample_df['Recently_Viewed_Product'] = my_sub['ProductID']
sample_df['Pageloads_last_7_days'] = my_sub['Pageloads_last_7_days']
sample_df['Clicks_last_7_days'] = my_sub['Clicks_last_7_days']

sample_df.to_csv('V.RoughSub1.csv', index=False)

In [45]:
sample_df['Most_Viewed_product_15_Days'] = combined_df['ProductID'].mode()[0]
sample_df['Most_Active_OS'] = combined_df['OS'].mode()[0]
sample_df['Recently_Viewed_Product'] = combined_df['ProductID'].mode()[0]
sample_df['Pageloads_last_7_days'] = my_sub['Pageloads_last_7_days']
sample_df['Clicks_last_7_days'] = my_sub['Clicks_last_7_days']
sample_df['No_of_days_Visited_7_Days'] = combined_df['days'].mean()

sample_df.to_csv('V.RoughSub1.csv', index=False)

In [41]:
sample_df.head(2)

,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days,Most_Active_OS,Recently_Viewed_Product,Pageloads_last_7_days,Clicks_last_7_days
0,U100002,1,1,1,Pr100258,Android,Product101,3.872105,12.293281
1,U100003,1,1,1,Pr100079,Windows,Product101,1.000000,2.000000


In [42]:
combined_df.head(2)

,webClientID,VisitDateTime,ProductID,UserID,Activity,Browser,OS,City,Country,newVisitDateTime,days,Signup Date,User Segment
0,WI100000112772,1526366895249000000,Pr100017,U106593,CLICK,Chrome Mobile,Android,,,2018-05-15 07:48:15,15.0,2017-09-10 15:48:09.451327+00:00,B
1,WI100000112772,2018-05-16 06:53:23.737,Pr100241,U106593,PAGELOAD,Chrome Mobile,Android,,,2018-05-16 06:53:23,16.0,2017-09-10 15:48:09.451327+00:00,B


### User_Vintage
	

Vintage (In Days) of the user as of today

In [157]:
combined_df.head(1)

,webClientID,VisitDateTime,ProductID,UserID,Activity,Browser,OS,City,Country,newVisitDateTime,days,Signup Date,User Segment
0,WI100000112772,1526366895249000000,Pr100017,U106593,CLICK,Chrome Mobile,Android,,,2018-05-15 07:48:15,15.0,2017-09-10 15:48:09.451327+00:00,B


In [115]:
sample_df.head()

,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days,Most_Active_OS,Recently_Viewed_Product,Pageloads_last_7_days,Clicks_last_7_days
0,U100002,1,1,1,P12345,OS12345,P12345,1,1
1,U100003,1,1,1,P12345,OS12345,P12345,1,1
2,U100004,1,1,1,P12345,OS12345,P12345,1,1
3,U100005,1,1,1,P12345,OS12345,P12345,1,1
4,U100006,1,1,1,P12345,OS12345,P12345,1,1
